1 - Find website to get all E-Commerces

2 - Get columns and data

3 - Connect to SQL and create SQL Objects for them

4 - Store in DataFrame

5 - Store DataFrame into DB

6 - Create reports for user to see periodically


In [74]:
from bs4 import BeautifulSoup
import requests
##import pandas as pd
import pyodbc
import sqlite3
from datetime import date
##Email sender
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText

In [75]:
##Email variables
sender_email = ""
receiver_email = ""
subject = "Alerta de preço baixo!"
message = MIMEMultipart()
message['From'] = sender_email
message['To'] = receiver_email
message['Subject'] = subject
##
current_date = date.today()
connection = sqlite3.connect('webscrapping.db')
cursor = connection.cursor()

url_hidratante = 'https://br.loccitaneaubresil.com/pt-br/p/geleia-hidratante-desodorante-corporal-roma/BROBVS100547.html'
page = requests.get(url_hidratante)
soup = BeautifulSoup(page.text, 'html.parser')

In [76]:
product_name1 = 'Geleia Hidratante Desodorante Corporal Romã' #soup.find_all('div', class_ = 'pdp-product-name')
product_price_place = soup.find_all('span', class_ = 'value')
product_price = 75##product_price_place[0].get('content') # Gets content from span value
product_price_converted = float(product_price)

cursor.execute('''
INSERT INTO ProductPrice (pdName, pdPrice, pdDate)
VALUES (?, ?, ?)
''', (product_name1, product_price_converted, current_date))

connection.commit()
connection.close()

if product_price_converted < 80:
    body = f"O valor de {product_name1} está abaixo de 80 reais, seu valor atualmente está em R$ {product_price_converted:.2f}, aproveite!"
    message.attach(MIMEText(body, 'plain'))
    #Email sending
    smtp_server = "smtp.gmail.com"
    smtp_port = 587
    password = "" 
    
    try:
        # Connect to the server and log in
        server = smtplib.SMTP(smtp_server, smtp_port)
        server.starttls()  # Secure the connection
        server.login(sender_email, password)
        
        # Send the email
        server.send_message(message)
        print("Email sent successfully!")
        
    except Exception as e:
        print(f"Failed to send email: {e}")
    finally:
        server.quit()


C:\Users\Vitor Leandro\AppData\Local\Temp\ipykernel_19652\3225391519.py:6: DeprecationWarning: The default date adapter is deprecated as of Python 3.12; see the sqlite3 documentation for suggested replacement recipes
  cursor.execute('''


Email sent successfully!
